# **HW3**

**Instructions:** Submit an .ipynb file to iCollege with all output generated and showing and code fully commented. (To download an .ipynb file from Colab, go to File >> Download.) I run all code in the homework and will not debug or troubleshoot if I get errors, so be sure to double check your work.

**Due date:** 9:00 a.m., March 2

In [1]:
# Import relevant packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns

import collections
from collections import Counter
from itertools import chain
import string
import spacy
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk import RegexpTokenizer
from nltk.text import Text
from wordcloud import WordCloud
from spacy.lang.en.examples import sentences 
from nltk.corpus import stopwords
from scipy import sparse, spatial
from scipy.sparse import csc_matrix
import gensim.models.keyedvectors as word2vec
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sentence_transformers import SentenceTransformer, util




nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Thanh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Thanh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Thanh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Thanh\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

1. Run all code from Classes 1 and 2, using the class code that is posted on iCollege, stopping at the **Tokenization** section header in Class 2. At that point, you should have a dataframe called dc_df with 285 rows and 12 columns, including a 'synopsis' column. The only cleaning that you should have done to the text is to remove the /n symbols as in Class 1. **Show the first five rows of the dc_df dataframe here and print the dataframe shape.** If your shape is not (285, 12), go back and re-run the Class 1 and 2 code exactly so that you are starting this homework from the right place. **You will use the 'synopsis' column as your text column for the remainder of this homework.**
> You can show me all of your Class 1 and 2 code here or you can run it separately, save the dc_df dataframe, and then import it here to begin this homework. Either is fine, as long as you print the first 5 rows and the dataframe shape here.
> Your 12 columns should be: 'filename', 'text', 'case_name', 'cite', 'date', 'court', 'state_fed', 'trial_appellate', 'jurisd_measure_enacted', 'text_length', 'synopsis', 'holdings'

In [2]:
#read in data and create dataframe
dc_df = pd.read_csv('C:/Users/Thanh/OneDrive/Documents/MBA/MSA 8350 - Text Analytics/dc_df.csv')

In [3]:
# Look at what's left in the 'text' column in the first row
[dc_df.iloc[0,1]] # Row 0, column 1 ('text')

['   454 F.Supp.3d 1188 United States District Court, M.D. Alabama, Northern Division. Yashica ROBINSON, M.D., et al., on behalf of themselves, their patients, physicians, clinic administrators, and staff, Plaintiffs, v. Steven MARSHALL, in his official capacity as Alabama Attorney General, et al., Defendants. CIVIL ACTION NO. 2:19cv365-MHT | Signed April 12, 2020 ']

In [4]:
# How many "None" values are in the 'synopsis' column?
dc_df.isna().sum()

filename                   0
text                       0
text_length                0
case_name                  0
cite                       0
date                       0
court                      0
state_fed                  0
trial_appellate            0
jurisd_measure_enacted     0
synopsis                   0
holdings                  15
dtype: int64

In [5]:
# Look at what's left in the 'synopsis' column in the first row
[dc_df.iloc[0,10]] # Row 0, column 10 ('synopsis')

[" Background: Abortion providers brought action against Alabama's attorney general challenging constitutionality of Alabama statute that imposed criminal liability on abortion providers for nearly all abortions, completed or attempted, regardless of fetal viability. Providers moved to preliminarily enjoin enforcement of state health officer's order mandating postponement of all non-emergency medical procedures during COVID-19 outbreak.   "]

In [6]:
dc_df.shape

(285, 12)

In [7]:
dc_df.columns

Index(['filename', 'text', 'text_length', 'case_name', 'cite', 'date', 'court',
       'state_fed', 'trial_appellate', 'jurisd_measure_enacted', 'synopsis',
       'holdings'],
      dtype='object')

In [8]:
dc_df

,filename,text,text_length,case_name,cite,date,court,state_fed,trial_appellate,jurisd_measure_enacted,synopsis,holdings
0,86-Robinson-v-Marshall.txt,454 F.Supp.3d 1188 United States District C...,67944,"Robinson v. Marshall, 454 F.Supp.3d 1188",454 F.Supp.3d 1188,04-12-2020,DC_Alabama Middle,federal,trial,Alabama,Background: Abortion providers brought action...,": The District Court, Myron H. Thompson, Senio..."
1,043-Children_s-Health-Def-v-Food-And-Drug-Admi...,573 F.Supp.3d 1234 United States District C...,28269,"Children's Health Defense v. FDA, 573 F.Supp.3...",573 F.Supp.3d 1234,11-30-2021,DC_Tennessee Eastern,federal,trial,Federal,"Background: Activist group, on behalf of its ...",": The District Court, Clifton L. Corker, J., h..."
2,074-Minnesota-Voters-Alliance-v-Walz.txt,494 F.Supp.3d 610 United States District Co...,10854,"Minnesota Voters Alliance v. Walz, 494 F.Supp....",494 F.Supp.3d 610,10-13-2020,DC_Minnesota,federal,trial,Minnesota,Background: Advocacy group and individual pol...,": The District Court, Patrick J. Schiltz, J., ..."
3,073-Minnesota-Voters-Alliance-v-Walz.txt,KeyCite Yellow Flag - Negative Treatment ...,56216,"Minnesota Voters Alliance v. Walz, 492 F.Supp....",492 F.Supp.3d 822,10-02-2020,DC_Minnesota,federal,trial,Minnesota,Background: Advocacy group and state resident...,": The District Court, Patrick J. Schiltz, J., ..."
4,006-ARC-of-Iowa-v-Reynolds.txt,KeyCite Yellow Flag - Negative Treatment ...,61065,"Arc of Iowa v. Reynolds, 559 F.Supp.3d 861",559 F.Supp.3d 861,09-13-2021,DC_Iowa Southern,federal,trial,Iowa,Background: Advocacy organization and parents...,": The District Court, Robert W. Pratt, J., hel..."
...,...,...,...,...,...,...,...,...,...,...,...,...
280,55-Air-Force-Officer-v-Austin_1.txt,KeyCite Yellow Flag - Negative Treatment ...,56026,"Air Force Officer v. Austin, et. al, 2022 WL 4...",2022 WL 468799,02-15-2022,DC_Georgia Middle,federal,trial,Federal,"Background: United States Air Force officer, ...",": The District Court, Tilman E. Self, III, J.,..."
281,15-Miller-v-Austin.txt,2022 WL 3584666 Only the Westlaw citation i...,20821,"Miller v. Austin, 2022 WL 3584666",2022 WL 3584666,08-22-2023,DC_Wyoming,federal,trial,Federal,Background: United States Air Force technical...,": The District Court, Scott W. Skavdahl, J., h..."
282,19-Colonel-Financial-Management-Officer-v-Aust...,KeyCite Blue Flag - Appeal Notification ...,88161,Colonol Financial Management Officer v. Austin...,2022 WL 3643512,08-18-2022,DC_Florida Middle,federal,trial,Federal,Background: United States Marine Corps servic...,": The District Court, Steven Merryday, J., hel..."
283,085-Georgia-v-Biden.txt,KeyCite Red Flag - Severe Negative Treatme...,63449,"Georgia v. Biden, 2021 WL 5779939\n",2021 WL 5779939,12-07-2021,DC_Georgia Southern,federal,trial,Federal,"Background: Various states, state officials, ...",": The District Court, R. Stan Baker, J., held ..."


2. Using **count vectors** and cosine similarity, identify the pair of synopses in the corpus that are the most similar to each other, but are not duplicates. Print the two synopses here using .iloc.
> For this question and Q3, you must use a *different* set of cleaning steps from the ones in the Class 7 code: (lowercase=True, stop_words='english', min_df=0.05). Explain your own cleaning choices in #comments in your homework code.  


In [9]:
#Using count vectors and cosine similarity, identify the pair of synopses in the corpus that are the most similar to each other, but are not duplicates. Print the two synopses here using .iloc.

# Import relevant packages

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse, spatial
from scipy.sparse import csc_matrix

# Create a CountVectorizer object
cv_vec = CountVectorizer(lowercase=True, stop_words='english', min_df=0.05)
# Create a sparse matrix with a row for each synopsis
cv = cv_vec.fit_transform(dc_df['synopsis'])

# Compute cosine similarity between every pair of vectors
cv_sim = cosine_similarity(cv, dense_output=False)

In [10]:
# View in pairs ranked by similarity
cv_pairs_coo = cv_sim.tocoo() # Convert to coordinate format  
text_1 = pd.DataFrame(cv_pairs_coo.row) # Create three sub-dataframes from coo rows
text_2 = pd.DataFrame(cv_pairs_coo.col) # columns
cosine_sim = pd.DataFrame(cv_pairs_coo.data) # data

# Conatenate into a single dataframe
cv_sim_df = pd.concat([text_1, text_2, cosine_sim], axis = 1) 
cv_sim_df.columns = ('text_1', 'text_2', 'cv_cosine_sim') # Rename columns
cv_sim_df = cv_sim_df.sort_values(by=['cv_cosine_sim'], ascending = False) # Sort
cv_sim_df.head(15)

,text_1,text_2,cv_cosine_sim
62405,218,218,1.0
51007,178,178,1.0
39323,137,137,1.0
9403,32,32,1.0
57652,202,202,1.0
80117,281,281,1.0
40911,143,143,1.0
33522,117,117,1.0
23863,83,83,1.0
67051,235,235,1.0


In [11]:
# Drop all self-matches; view again
cv_sim_df = cv_sim_df.drop(cv_sim_df[cv_sim_df['text_1'] == cv_sim_df['text_2']].index)
cv_sim_df = cv_sim_df.reset_index(drop = True) # Reset index
cv_sim_df = cv_sim_df.sort_values(by=['cv_cosine_sim'], ascending = False) # Sort
cv_sim_df.head(10)

,text_1,text_2,cv_cosine_sim
0,180,179,1.000000
1,179,180,1.000000
2,163,162,1.000000
3,162,163,1.000000
4,267,268,0.979611
5,268,267,0.979611
6,161,162,0.937437
9,163,161,0.937437
7,161,163,0.937437
8,162,161,0.937437


In [12]:
# Drop duplicate pairs (every other row)
cv_sim_df=cv_sim_df[pd.DataFrame(np.sort(cv_sim_df[['text_1','text_2']].values,1)).duplicated()]
cv_sim_df = cv_sim_df.reset_index(drop = True) # Reset index

C:\Users\Thanh\AppData\Local\Temp\ipykernel_15736\1997076595.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  cv_sim_df=cv_sim_df[pd.DataFrame(np.sort(cv_sim_df[['text_1','text_2']].values,1)).duplicated()]


In [13]:
# Look at table, sorted by cosine_sim highest to lowest
cv_sim_df = cv_sim_df.sort_values(by=['cv_cosine_sim'], ascending = False) # Sort
cv_sim_df.head(10)

,text_1,text_2,cv_cosine_sim
0,179,180,1.000000
1,162,163,1.000000
2,268,267,0.979611
4,162,161,0.937437
3,163,161,0.937437
5,205,204,0.893449
6,216,217,0.881794
7,128,18,0.828173
8,156,157,0.827535
9,149,148,0.819194


In [14]:
# Investigate some pairs
dc_df.iloc[[156,155]] # Row 0, column 10 ('synopsis')

,filename,text,text_length,case_name,cite,date,court,state_fed,trial_appellate,jurisd_measure_enacted,synopsis,holdings
156,028-Andre-Rodney-v-Hochul_1.txt,569 F.Supp.3d 128 United States District Co...,41621,"Andre-Rodney v. Hochul, 569 F.Supp.3d 128\n",569 F.Supp.3d 128,11-01-2021,DC_New York Northern,federal,trial,New York,Background: New York State security officers ...,": The District Court, Brenda K. Sannes, J., he..."
155,36-Broecker-v-New-York-City-Department-of-Educ...,585 F.Supp.3d 299 United States District Co...,69959,Broecker v. New York City Department of Educat...,2022 WL 426113,02-11-2022,DC_New York Eastern,federal,trial,New York,Background: New York City Department of Educa...,": The District Court, Kiyo A. Matsumoto, J., h..."


3. Using **TF-IDF vectors** and cosine similarity, identify the pair of synopses in the corpus that are the most similar to each other, but are not duplicates. Print the two synopses here using .iloc.
> See note above re: cleaning.

In [15]:

from sklearn.feature_extraction.text import TfidfVectorizer

# Create a TfidfVectorizer object
tfidf_vec = TfidfVectorizer(lowercase=True, stop_words='english', min_df=0.05)

# Create a sparse matrix with a row for each synopsis
tfidf = tfidf_vec.fit_transform(dc_df['synopsis'])

# Compute cosine similarity between every pair of vectors
tfidf_sim = cosine_similarity(tfidf, dense_output=False)

# View in pairs ranked by similarity
tfidf_pairs_coo = tfidf_sim.tocoo() # Convert to coordinate format
text_1 = pd.DataFrame(tfidf_pairs_coo.row) # Create three sub-dataframes from coo rows
text_2 = pd.DataFrame(tfidf_pairs_coo.col) # columns
cosine_sim = pd.DataFrame(tfidf_pairs_coo.data) # data

# Conatenate into a single dataframe
tfidf_sim_df = pd.concat([text_1, text_2, cosine_sim], axis = 1)
tfidf_sim_df.columns = ('text_1', 'text_2', 'tfidf_cosine_sim') # Rename columns
tfidf_sim_df = tfidf_sim_df.sort_values(by=['tfidf_cosine_sim'], ascending = False) # Sort
tfidf_sim_df.head(15)

# Drop all self-matches; view again
tfidf_sim_df = tfidf_sim_df.drop(tfidf_sim_df[tfidf_sim_df['text_1'] == tfidf_sim_df['text_2']].index)
tfidf_sim_df = tfidf_sim_df.reset_index(drop = True) # Reset index
tfidf_sim_df = tfidf_sim_df.sort_values(by=['tfidf_cosine_sim'], ascending = False) # Sort
tfidf_sim_df.head(3)

# Drop duplicate pairs (every other row)
tfidf_sim_df=tfidf_sim_df[pd.DataFrame(np.sort(tfidf_sim_df[['text_1','text_2']].values,1)).duplicated()]
tfidf_sim_df = tfidf_sim_df.reset_index(drop = True) # Reset index

# Look at table, sorted by cosine_sim highest to lowest
tfidf_sim_df = tfidf_sim_df.sort_values(by=['tfidf_cosine_sim'], ascending = False) # Sort
tfidf_sim_df.head(100)


C:\Users\Thanh\AppData\Local\Temp\ipykernel_15736\3943360727.py:31: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tfidf_sim_df=tfidf_sim_df[pd.DataFrame(np.sort(tfidf_sim_df[['text_1','text_2']].values,1)).duplicated()]


,text_1,text_2,tfidf_cosine_sim
0,162,163,1.000000
1,180,179,1.000000
2,267,268,0.983897
3,163,161,0.953971
4,161,163,0.953971
...,...,...,...
95,214,231,0.622879
96,67,65,0.622342
97,48,46,0.622298
98,124,260,0.622234


In [16]:

# Investigate some pairs
dc_df.iloc[[156,155]] # Row 0, column 10 ('synopsis')


,filename,text,text_length,case_name,cite,date,court,state_fed,trial_appellate,jurisd_measure_enacted,synopsis,holdings
156,028-Andre-Rodney-v-Hochul_1.txt,569 F.Supp.3d 128 United States District Co...,41621,"Andre-Rodney v. Hochul, 569 F.Supp.3d 128\n",569 F.Supp.3d 128,11-01-2021,DC_New York Northern,federal,trial,New York,Background: New York State security officers ...,": The District Court, Brenda K. Sannes, J., he..."
155,36-Broecker-v-New-York-City-Department-of-Educ...,585 F.Supp.3d 299 United States District Co...,69959,Broecker v. New York City Department of Educat...,2022 WL 426113,02-11-2022,DC_New York Eastern,federal,trial,New York,Background: New York City Department of Educa...,": The District Court, Kiyo A. Matsumoto, J., h..."


4. Using **word2vec word embeddings** and cosine similarity, identify the pair of synopses in the corpus that are the most similar to each other, but are not duplicates. Print the two synopses here using .iloc.

In [17]:

import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.test.utils import datapath


In [18]:

# Load the model
w2v_vectors = KeyedVectors.load_word2vec_format(datapath('C:/Users/Thanh/Downloads/GoogleNews-vectors-negative300.bin.gz'), binary=True)


In [19]:

# Save max length for use downstream
max_len = dc_df['synopsis'].str.len().max()
max_len

# Run keras tokenizer 
tokenizer = Tokenizer()
w2v_tokenized = tokenizer.fit_on_texts(dc_df['synopsis'])
w2v_tokenized_documents = tokenizer.texts_to_sequences(dc_df['synopsis'])
w2v_tokenized_padded_documents = pad_sequences(w2v_tokenized_documents, maxlen = max_len, padding = 'post') 


In [20]:
# Set up embedding matrix
vocab_size=len(tokenizer.word_index)+1 # Pulls from keras vectorization workflow above
embedding_matrix = np.zeros((vocab_size,300))

In [21]:
# Create word embeddings
for word,i in tokenizer.word_index.items():
    if word in w2v_vectors:
        embedding_matrix[i]=w2v_vectors[word] 

In [22]:
# Create document-level embeddings
document_word_embeddings = np.zeros((len(w2v_tokenized_padded_documents),max_len,300)) # Uses max_len from above

for i in range(len(w2v_tokenized_padded_documents)):
    for j in range(len(w2v_tokenized_padded_documents[0])):
        document_word_embeddings[i][j]=embedding_matrix[w2v_tokenized_padded_documents[i][j]]

In [23]:
# Print shape
document_word_embeddings.shape

(285, 90383, 300)

In [24]:
# Reduce dimensionality, preserving axes 0 (rows) and 2 (words) and averaging axis 1 (300 dimensions)
document_word_embeddings_avg = np.average(document_word_embeddings, axis=2)
document_word_embeddings_avg.shape 

(285, 90383)

In [25]:
# Compute cosine similarity
w2v_sim = cosine_similarity(document_word_embeddings_avg) 
w2v_sim = sparse.csr_matrix(w2v_sim) # Convert into sparse matrix

In [26]:
# View in pairs ranked by similarity
w2v_pairs_coo = w2v_sim.tocoo() # Convert to coordinate format  
text_1 = pd.DataFrame(w2v_pairs_coo.row) # Create three sub-dataframes from coo rows
text_2 = pd.DataFrame(w2v_pairs_coo.col) # columns
cosine_sim = pd.DataFrame(w2v_pairs_coo.data) # data

# Conatenate into a single dataframe
w2v_sim_df = pd.concat([text_1, text_2, cosine_sim], axis = 1) 
w2v_sim_df.columns = ('text_1', 'text_2', 'w2v_cosine_sim') # Rename columns
w2v_sim_df = w2v_sim_df.sort_values(by=['w2v_cosine_sim'], ascending = False) # Sort

# Drop all self-matches
w2v_sim_df = w2v_sim_df.drop(w2v_sim_df[w2v_sim_df['text_1'] == w2v_sim_df['text_2']].index)
w2v_sim_df = w2v_sim_df.reset_index(drop = True) # Reset index
w2v_sim_df = w2v_sim_df.sort_values(by=['w2v_cosine_sim'], ascending = False) # Sort

# Drop duplicate pairs (every other row)
w2v_sim_df = w2v_sim_df.iloc[::2, :]
w2v_sim_df = w2v_sim_df.reset_index(drop = True) # Reset index

# Look at table, sorted by cosine_sim highest to lowest
w2v_sim_df = w2v_sim_df.sort_values(by=['w2v_cosine_sim'], ascending = False) # Sort
w2v_sim_df.head(10)

,text_1,text_2,w2v_cosine_sim
0,179,180,1.000000
1,163,162,1.000000
2,161,162,0.980411
3,161,163,0.980411
4,267,268,0.960939
5,148,149,0.918584
6,216,217,0.702510
7,171,106,0.660420
8,216,215,0.630508
9,87,88,0.628342


In [27]:
#compare top two text of w2v_consine_sim pairs from each model

print(dc_df['synopsis'].iloc[w2v_sim_df['text_1'].iloc[2]])
print(dc_df['synopsis'].iloc[w2v_sim_df['text_2'].iloc[2]])



 Background: Officer in the United States Marine Corps (USMC) brought action alleging denial of his religious exemption from COVID-19 vaccine mandate violated the Religious Freedom Restoration Act (RFRA). Officer filed motion for preliminary injunction and temporary restraining order (TRO).   
 Background: Officer in the United States Marine Corps (USMC) brought action alleging denial of his religious exemption from COVID-19 vaccine mandate violated the Religious Freedom Restoration Act (RFRA). Officer filed motion for preliminary injunction.   


5. Using **sentence embeddings from Sentence Transformer** and cosine similarity, identify the pair of synopses in the corpus that are the most similar to each other, but are not duplicates. Print the two synopses here using .iloc.

In [28]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [29]:
#Using sentence embeddings from Sentence Transformer and cosine similarity, identify the pair of synopses in the corpus that are the most similar to each other, but are not duplicates. Print the two synopses here using .iloc.

sentences = dc_df['synopsis'].tolist() # Convert to list
# Load the model
transformer_encoded = model.encode(sentences)

In [30]:
transformer_sim = cosine_similarity(transformer_encoded) 
transformer_sim = sparse.csr_matrix(transformer_sim) # Convert into sparse matrix

In [31]:
# View in pairs ranked by similarity
transformer_pairs_coo = transformer_sim.tocoo() # Convert to coordinate format  
text_1 = pd.DataFrame(transformer_pairs_coo.row) # Create three sub-dataframes from coo rows
text_2 = pd.DataFrame(transformer_pairs_coo.col) # columns
cosine_sim = pd.DataFrame(transformer_pairs_coo.data) # data

# Conatenate into a single dataframe
transformer_sim_df = pd.concat([text_1, text_2, cosine_sim], axis = 1) 
transformer_sim_df.columns = ('text_1', 'text_2', 'transformer_cosine_sim') # Rename columns
transformer_sim_df = transformer_sim_df.sort_values(by=['transformer_cosine_sim'], ascending = False) # Sort

# Drop all self-matches
transformer_sim_df = transformer_sim_df.drop(transformer_sim_df[transformer_sim_df['text_1'] == transformer_sim_df['text_2']].index)
transformer_sim_df = transformer_sim_df.reset_index(drop = True) # Reset index
transformer_sim_df = transformer_sim_df.sort_values(by=['transformer_cosine_sim'], ascending = False) # Sort

# Drop duplicate pairs (every other row)
transformer_sim_df = transformer_sim_df.iloc[::2, :]
transformer_sim_df = transformer_sim_df.reset_index(drop = True) # Reset index

# Look at table, sorted by cosine_sim highest to lowest
transformer_sim_df = transformer_sim_df.sort_values(by=['transformer_cosine_sim'], ascending = False) # Sort
transformer_sim_df.head(10)


,text_1,text_2,transformer_cosine_sim
0,180,179,1.000000
1,162,163,1.000000
2,268,267,0.990857
3,204,205,0.980348
4,148,149,0.947909
5,163,161,0.945507
6,161,162,0.945507
7,215,216,0.932624
8,88,87,0.930345
9,244,247,0.920769


In [36]:
#compare top two text pairs from each model
print(dc_df['synopsis'].iloc[transformer_sim_df['text_1'].iloc[0]])
print(dc_df['synopsis'].iloc[transformer_sim_df['text_2'].iloc[0]])


 Background: Parents of 12 public school students who suffered from disabilities or illnesses brought action against Commonwealth of Virginia, its Governor, Attorney General and other related parties, seeking to enjoin defendants from enforcing Executive Order and related legislation permitting parents to opt their children out of masking requirements in Virginia's schools, even if school or district otherwise determined mandatory masking to be an appropriate COVID-19 prevention measure.   
 Background: Parents of 12 public school students who suffered from disabilities or illnesses brought action against Commonwealth of Virginia, its Governor, Attorney General and other related parties, seeking to enjoin defendants from enforcing Executive Order and related legislation permitting parents to opt their children out of masking requirements in Virginia's schools, even if school or district otherwise determined mandatory masking to be an appropriate COVID-19 prevention measure.   


6. If your answers to Q2-Q5 are different from one another, explain why. If they are the same, explain why.

In [35]:
#merge the 4 columns transformer_cosine_sim, w2v_cosine_sim,cv_cosine_sim columns into one dataframe
merged_df = pd.concat([transformer_sim_df, w2v_sim_df, cv_sim_df, tfidf_sim_df], axis = 1)
merged_df.head(25)



,text_1,text_2,transformer_cosine_sim,text_1,text_2,w2v_cosine_sim,text_1,text_2,cv_cosine_sim,text_1,text_2,tfidf_cosine_sim
0,180,179,1.000000,179,180,1.000000,179,180,1.000000,162,163,1.000000
1,162,163,1.000000,163,162,1.000000,162,163,1.000000,180,179,1.000000
2,268,267,0.990857,161,162,0.980411,268,267,0.979611,267,268,0.983897
3,204,205,0.980348,161,163,0.980411,163,161,0.937437,163,161,0.953971
4,148,149,0.947909,267,268,0.960939,162,161,0.937437,161,163,0.953971
5,163,161,0.945507,148,149,0.918584,205,204,0.893449,205,204,0.893344
6,161,162,0.945507,216,217,0.702510,216,217,0.881794,217,216,0.871996
7,215,216,0.932624,171,106,0.660420,128,18,0.828173,148,149,0.860243
8,88,87,0.930345,216,215,0.630508,156,157,0.827535,266,265,0.798165
9,244,247,0.920769,87,88,0.628342,149,148,0.819194,98,88,0.794323


Looking at the merged dataframe above the top index [180 179] for all 4 models have same cosine similarity of 1 because the texts are identical, anything above .9 in cosine sim has similar synopsis, besides this the weight vary between all the models. As you can see Transformer outperfrom the other 3 based on the cosine score alone. Because transformer-based models, take into account contextual information, which can lead to more accurate vector representations for words or documents. This contextual information might not be captured by simpler techniques like count vectorization or TF-IDF, leading to differences in cosine similarity values. While word2vec cosine simialrity might be trained on a different corpus compared to transformed based consine similarity model.